# Imports

In [59]:
import re
import os
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion
import importlib.util
from pathlib import Path

# Parse Input

In [65]:
class InputData:
    def __init__(self, input_text: Path):
        self.parse(input_text)
        self.full_description = f"{self.signature}\n{self.description}"
    
    def __repr__(self) -> str:
        return f"function_name: {self.function_name}\nsignature: {self.signature}\ndescription: {self.description}\nexamples: {self.examples}\ntemperatures: {self.temperatures}\nnum: {self.num}\nunit_tests: {self.unit_tests}"
    
    def parse(self, input_text: Path):
        with open(input_text, 'r') as f:
            self.text = f.readlines()
        self.text = [line.strip().removesuffix(";") for line in self.text if line.strip() != '']
        self.function_name = self.text[0]
        self.description = ""
        for i, line in enumerate(self.text[2:]):
            if '"""' in line:
                break
            if 'signature' in line:
                self.signature = line
            else:
                self.description += f"{line} "
        
        next_part = self.text[i + 1:]
        self.examples = []
        for i, line in enumerate(next_part):
            if "example:" in line:
                self.examples.append(line)
            if "temperatures:" in line:
                self.temperatures = line.split(":")[1].split(",")
                self.temperatures = [float(temp) for temp in self.temperatures]
            if "num" in line:
                self.num = int(line.split('=')[1])
            if "unit_tests" in line:
                self.unit_tests = next_part[i + 1:]
                break
            
            


In [66]:
input_text = Path('input.txt')
input_data = InputData(input_text)
input_data

function_name: sqrt_list
signature: signature: sqrt_list(l:list) -> float
description: A function named sqrt_list that takes a list of integer and returns the square root of the sum of that list, to two decimal places. 
examples: ['example: sqrt_list([1,7,8]) -> 4.0']
temperatures: [0.0, 0.8]
num: 3
unit_tests: ['sqrt_list([6,18,7,5]) -> 6.0', 'sqrt_list([50,2,33,8,4,3,10]) -> 10.48']

# Intigrate model

In [62]:
AZURE_OPENAI_API_KEY="e865ffe4387f416ea57d7394ee09fb2e"
AZURE_OPENAI_ENDPOINT="https://openaifor3267.openai.azure.com/"
AZURE_OPENAI_API_VERSION = "2023-12-01-preview"
model = "gpt-35-16k"

In [63]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT
)

In [64]:
def sent_prompt(prompt: str, temp: float, num_of_samples: int) -> ChatCompletion:
    response = client.chat.completions.create(
        model=model,
        temperature=temp,
        n=num_of_samples,
        max_tokens=100,
        messages=[
            {"role": "system", "content": """Act as experienced python developer,
            and provide the implementation for the following functions, make sure to only provide the implementation for the function and not
            put any redundant text as comment, the function should compile without any interventions."""},
            {"role": "user", "content": prompt}
        ]
    )
    return response

In [67]:
responses = []
for temp in input_data.temperatures:
    response_without_ex = {}
    response = sent_prompt(input_data.full_description, temp, input_data.num)
    response_without_ex["temp"] = temp
    response_without_ex["isExample"] = False
    response_without_ex["response"] = response
    responses.append(response_without_ex)

    response_with_ex = {}
    prompt = input_data.full_description + "\n" + "\n".join(input_data.examples)
    response = sent_prompt(prompt, temp, input_data.num)
    response_with_ex["temp"] = temp
    response_with_ex["isExample"] = True
    response_with_ex["response"] = response
    responses.append(response_with_ex)

In [76]:
responses

[{'temp': 0.0,
  'isExample': False,
  'response': ChatCompletion(id='chatcmpl-9FipqKYlSk2vrpjenZyOQ8VzQGBhy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='import math\n\ndef sqrt_list(l):\n    return round(math.sqrt(sum(l)), 2)', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}), Choice(finish_reason='stop', index=1, logprobs=None, message=ChatCompletionMessage(content='import math\n\ndef sqrt_list(l):\n    return round(math.sqrt(sum(l)), 2)', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered'

# Generate Files

In [80]:
file_names = []
for result in responses:
    for i, choice in enumerate(result['response'].choices):
        file_name = f"{input_data.function_name}-{'ex-' if result['isExample'] else ''}s{i + 1}-t{int(result['temp'] * 10)}.py"
        with open(file_name, 'w') as f:
            f.write(choice.message.content)
        print(f"File {file_name} created")
        file_names.append(file_name)

File sqrt_list-s1-t0.py created
File sqrt_list-s2-t0.py created
File sqrt_list-s3-t0.py created
File sqrt_list-ex-s1-t0.py created
File sqrt_list-ex-s2-t0.py created
File sqrt_list-ex-s3-t0.py created
File sqrt_list-s1-t8.py created
File sqrt_list-s2-t8.py created
File sqrt_list-s3-t8.py created
File sqrt_list-ex-s1-t8.py created
File sqrt_list-ex-s2-t8.py created
File sqrt_list-ex-s3-t8.py created


# Add Tests and gather results